In [3]:
from dataloader import CustomImageDataset, DataLoader
from torchvision.transforms import Compose
import torchvision.transforms as transforms

transform = Compose([
    transforms.ToTensor(), # Scales data into [0,1]
    transforms.Lambda(lambda t: (t * 2) - 1) # Scale between [-1, 1]
])

dataset = CustomImageDataset(annotations_dir = '/home/pqbas/dl/detection/MNIST-ObjectDetection/data/mnist_detection/test/labels',
                             img_dir = '/home/pqbas/dl/detection/MNIST-ObjectDetection/data/mnist_detection/test/images',
                             data_transform = transform,
                             size=(448,448))

train_loader = DataLoader(dataset, batch_size=5, shuffle=True)


torch.Size([5, 1, 448, 448])
torch.Size([5, 7, 7, 16])


In [5]:
import torch
import torch.nn as nn
from metrics import intersection_over_union
from model import YOLOv1

model = YOLOv1(B=2, C=10, S=7)

mse = nn.MSELoss(reduction='sum')

n_break = 0
for idx,(img, target) in enumerate(train_loader):
    
    one_obj = one_obj[...,0][...,None]
    
    prediction = model(img)

    # ========================== #
    #        Object Loss         #
    # ========================== #

    target_box = target[...,0:4]

    box1 = torch.sigmoid(prediction[...,0:4])
    box2 = torch.sigmoid(prediction[...,4:8])

    iou1 = intersection_over_union(box1, target_box)
    iou2 = intersection_over_union(box2, target_box)
    
    ious = torch.cat([iou1,iou2], dim=3)
    max, bestbox = torch.max(ious, dim=3)
    
    box = (bestbox[...,None]*box2 +  (1 - bestbox[...,None])*box1) * one_obj

    box[...,2:4] = torch.sqrt(box[...,2:4])
    target[...,2:4] = torch.sqrt(target_box[...,2:4])

    box_coordinates_loss = mse(box,target_box)
    
    # ========================== #
    #        Classf Loss         #
    # ========================== #

    pred_prob = prediction[...,10:21]*one_obj
    target_prob = target[...,4:15]*one_obj

    classf_loss = mse(pred_prob, target_prob)


    # ========================= #
    #        Noobj Loss         #
    # ========================= #

    noobj = 1 - one_obj
    noobj_pred_prob = prediction[...,10:21]*noobj
    noobj_target_prob = target[...,4:15]*noobj

    print(noobj_pred_prob.shape)
    print(noobj_target_prob.shape)

    mse(noobj_pred_prob, noobj_target_prob)


    if idx == n_break:
        break

NameError: name 'one_obj' is not defined